In [1]:
from azureml.core import Workspace, Dataset, Experiment
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import joblib

## Training the LR model

In [2]:
from azureml.core import Dataset, Workspace

# Load workspace
ws = Workspace.from_config()

# Get the dataset
dataset = Dataset.get_by_name(ws, name='diabetes-data')

# Convert to pandas DataFrame
df = dataset.to_pandas_dataframe()

# Display the first few rows
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
# Split data
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, 'model.pkl')

['model.pkl']

## Registering the LR Model 

In [4]:
from azureml.core import Model

# Register the model
model = Model.register(
    workspace=ws,
    model_path="model.pkl",
    model_name="diabetes-lr",
    tags={"framework": "scikit-learn"}
)

Registering model diabetes-lr


In [5]:
# Checking the avialable env
for env in ws.environments:
    print(env)

AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu
AzureML-Pytorch1.7-Cuda11-OpenMpi4.1.0-py36
AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36
AzureML-TensorFlow2.4-Cuda11-OpenMpi4.1.0-py36


In [7]:
# Creating a custom environment
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment(name="diabetets-env")

# Add dependencies
conda_deps = CondaDependencies()
conda_deps.add_pip_package("scikit-learn")
conda_deps.add_pip_package("joblib")
conda_deps.add_pip_package("numpy")

# Assigning dependencies to env
env.python.conda_dependencies = conda_deps

env.register(workspace=ws)

{
    "assetId": "azureml://locations/northeurope/workspaces/7d873530-0dcb-4b64-849b-903f34f77508/environments/diabetets-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240908.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "diabetets-env",
   

## Deploy the Model to ACI

In [12]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Inference configuration
inf_config = InferenceConfig(
    entry_script="score.py",  # Ensure this is a .py file, not .ipynb
    environment=env  # Use the custom environment
)

# Deployment config
deploy_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1
)

# Model deployment
service = Model.deploy(
    workspace=ws,
    name="diabetes-aci-service-v2",
    models=[model],
    inference_config=inf_config,
    deployment_config=deploy_config
)

service.wait_for_deployment(show_output=True)
print(service.scoring_uri)

/tmp/ipykernel_3113/2235063264.py:17: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-02-19 09:28:53+00:00 Creating Container Registry if not exists.
2025-02-19 09:28:54+00:00 Building image..
2025-02-19 09:35:00+00:00 Generating deployment configuration..
2025-02-19 09:35:01+00:00 Submitting deployment to compute..
2025-02-19 09:35:08+00:00 Checking the status of deployment diabetes-aci-service-v2..
2025-02-19 09:36:35+00:00 Checking the status of inference endpoint diabetes-aci-service-v2.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://8fb3b137-7061-4621-a971-2bb424fc12d6.northeurope.azurecontainer.io/score


## Testing the deployed model

In [14]:
import requests
import json

# Input data for prediction
input_data = json.dumps({
    "data": [[6, 148, 72, 35, 0, 33.6, 0.627, 50]]  # Example input
})

# Make a POST request
response = requests.post(
    url=service.scoring_uri,
    data=input_data,
    headers={"Content-Type": "application/json"}
)

# Print the prediction result
print(response.json())

{"prediction": [1]}
